### 语义检索，利用Embedding优化你的搜索功能

In [1]:

import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")
COMPLETION_MODEL = "text-davinci-003"

def generate_data_by_prompt(prompt):
    response = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    return response.choices[0].text

prompt = """请你生成50条淘宝网里的商品的标题，每条在30个字左右，品类是3C数码产品，标题里往往也会有一些促销类的信息，每行一条。"""
data = generate_data_by_prompt(prompt)

In [2]:

import pandas as pd

product_names = data.strip().split('\n')
df = pd.DataFrame({'product_name': product_names})
df.head()

,product_name
0,1. 【新款】Apple/苹果 iPhone XR 128G 全网通手机
1,2. 【热销】华为 Mate20 Pro 8GB+128GB 全网通智能手机
2,3. 【特惠】小米9 6GB+128GB 全网通智能手机
3,4. 【新品】荣耀V20 6GB+128GB 全网通智能手机
4,5. 【抢购】OPPO Reno 10GB+256GB 全网通智能手机


In [3]:

df.product_name = df.product_name.apply(lambda x: x.split('.')[1].strip())
df.head()

,product_name
0,【新款】Apple/苹果 iPhone XR 128G 全网通手机
1,【热销】华为 Mate20 Pro 8GB+128GB 全网通智能手机
2,【特惠】小米9 6GB+128GB 全网通智能手机
3,【新品】荣耀V20 6GB+128GB 全网通智能手机
4,【抢购】OPPO Reno 10GB+256GB 全网通智能手机


In [4]:

clothes_prompt = """请你生成50条淘宝网里的商品的标题，每条在30个字左右，品类是女性的服饰箱包等等，标题里往往也会有一些促销类的信息，每行一条。"""
clothes_data = generate_data_by_prompt(clothes_prompt)
clothes_product_names = clothes_data.strip().split('\n')
clothes_df = pd.DataFrame({'product_name': clothes_product_names})
clothes_df.product_name = clothes_df.product_name.apply(lambda x: x.split('.')[1].strip())
clothes_df.head()

,product_name
0,【新款】潮流百搭时尚女士包包
1,【折扣】精致简约时尚双肩背包
2,【特惠】潮流拼接时尚女士包
3,【热卖】经典百搭时尚女士手提包
4,【限量】潮流水洗棉时尚女士钱包


In [5]:

df = pd.concat([df, clothes_df], axis=0)
df = df.reset_index(drop=True)
display(df)

,product_name
0,【新款】Apple/苹果 iPhone XR 128G 全网通手机
1,【热销】华为 Mate20 Pro 8GB+128GB 全网通智能手机
2,【特惠】小米9 6GB+128GB 全网通智能手机
3,【新品】荣耀V20 6GB+128GB 全网通智能手机
4,【抢购】OPPO Reno 10GB+256GB 全网通智能手机
...,...
95,【折扣】简约时尚女士手提包
96,【特惠】经典复古时尚女士钱包
97,【热卖】潮流水洗棉时尚女士双肩背包
98,【限量】精致简约时尚女士手拿包


In [6]:

from openai.embeddings_utils import get_embeddings
import openai, os, backoff

openai.api_key = os.environ.get("OPENAI_API_KEY")
embedding_model = "text-embedding-ada-002"

batch_size = 100

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embeddings_with_backoff(prompts, engine):
    embeddings = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        embeddings += get_embeddings(list_of_text=batch, engine=engine)
    return embeddings

prompts = df.product_name.tolist()
prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

embeddings = []
for batch in prompt_batches:
    batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
    embeddings += batch_embeddings

df["embedding"] = embeddings
df.to_parquet("data/taobao_product_title.parquet", index=False)

In [7]:

from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the reviews for a specific product
def search_product(df, query, n=3, pprint=True):
    product_embedding = get_embedding(
        query,
        engine=embedding_model
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .product_name
    )
    if pprint:
        for r in results:
            print(r)
    return results

results = search_product(df, "自然淡雅背包", n=3)

【折扣】精致简约时尚双肩背包
【折扣】精致简约时尚女士双肩背包
【特惠】精致简约时尚女士双肩背包


In [8]:

def recommend_product(df, product_name, n=3, pprint=True):
    product_embedding = df[df['product_name'] == product_name].iloc[0].embedding
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .product_name
    )
    if pprint:
        for r in results:
            print(r)
    return results

results = recommend_product(df, "【限量】OnePlus 7T Pro 8GB+256GB 全网通", n=3)

IndexError: single positional indexer is out-of-bounds

In [10]:

import faiss
import numpy as np

def load_embeddings_to_faiss(df):
    embeddings = np.array(df['embedding'].tolist()).astype('float32')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index

index = load_embeddings_to_faiss(df)

In [11]:

def search_index(index, df, query, k=5):
    query_vector = np.array(get_embedding(query, engine=embedding_model)).reshape(1, -1).astype('float32')
    distances, indexes = index.search(query_vector, k)

    results = []
    for i in range(len(indexes)):
        product_names = df.iloc[indexes[i]]['product_name'].values.tolist()
        results.append((distances[i], product_names))    
    return results

products = search_index(index, df, "自然淡雅背包", k=3)

for distances, product_names in products:
    for i in range(len(distances)):
        print(product_names[i], distances[i])

【折扣】精致简约时尚双肩背包 0.20400454
【折扣】精致简约时尚女士双肩背包 0.20887718
【特惠】精致简约时尚女士双肩背包 0.20942491
